# Election Configuration

An election in ElectionGuard is defined as a set of metadata and cryptographic artifacts necessary to encrypt, conduct, tally, decrypt, and verify an election. The Data format used for election metadata is based on the [NIST Election Common Standard Data Specification](https://www.nist.gov/itl/voting/interoperability) but includes some modifications to support the end-to-end cryptography of ElectionGuard.

Election metadata is described in a specific format parseable into an `Manifest` and it's validity is checked to ensure that it is of an appropriate structure to conduct an End-to-End Verified ElectionGuard Election. ElectionGuard only verifies the components of the election metadata that are necessary to encrypt and decrypt the election. Some components of the election metadata are not checked for structural validity, but are used when generating a hash representation of the `Manifest`.

From an `Manifest` we derive an `InternalManifest` that includes a subset of the elements from the `Manifest` required to verify ballots are correct. Additionally a `CiphertextElectionContext` is created during the [Key Ceremony](/1_Key_Ceremony.md) that includes the cryptographic artifacts necessary for encrypting ballots.

## Glossary

- **Election Manifest** The election metadata in json format that is parsed into an Election Description
- **Election Description** The election metadata that describes the structure and type of the election, including geopolitical units, contests, candidates, and ballot styles, etc.
- **Internal Election Description** The subset of the `Manifest` required by ElectionGuard to validate ballots are correctly associated with an election. This component mutates the state of the Election Description.
- **Ciphertext Election Context** The cryptographic context of an election that is configured during the `Key Ceremony`
- **Description Hash** a Hash representation of the original Manifest.

## Process

1. Define an election according to the `Manifest` requirements.
2. Use the [NIST Common Standard Data Specification](https://www.nist.gov/itl/voting/interoperability) as a guide, but note the differences in [election.py](https://github.com/microsoft/electionguard-python/tree/main/src/electionguard.election.py) and the provided [sample manifest](https://github.com/microsoft/electionguard-python/tree/main/data/election_manifest_simple.json).
3. Parse the `Manifest` into the application.
4. Define the encryption parameters necessary for conducting an election (see `Key Ceremony`).
5. Create the Pubic Key either from a single secret, or from the Key Ceremony.
6. Build the `InternalManifest` and `CiphertextElectionContext` from the `Manifest` and `ElGamalKeyPair.public_key`.

## Usage Example

In [ ]:

import os
from electionguard.election import CiphertextElectionContext
from electionguard.election_builder import ElectionBuilder
from electionguard.elgamal import ElGamalKeyPair, elgamal_keypair_from_secret
from electionguard.manifest import Manifest, InternalManifest

# Open an election manifest file
with open(os.path.join(some_path, "election-manifest.json"), "r") as manifest:
        string_representation = manifest.read()
        election_description = Manifest.from_json(string_representation)

# Create an election builder instance, and configure it for a single public-private keypair.
# in a real election, you would configure this for a group of guardians.  See Key Ceremony for more information.
builder = ElectionBuilder(
    number_of_guardians=1,     # since we will generate a single public-private keypair, we set this to 1
    quorum=1,                  # since we will generate a single public-private keypair, we set this to 1
    description=election_description
)

# Generate an ElGamal Keypair from a secret.  In a real election you would use the Key Ceremony instead.
some_secret_value: int = 12345
keypair: ElGamalKeyPair = elgamal_keypair_from_secret(some_secret_value)

builder.set_public_key(keypair.public_key)

# get an `InternalElectionDescription` and `CiphertextElectionContext`
# that are used for the remainder of the election.
(internal_manifest, context) = builder.build()

## Constants

The election constants are the four constants that sit underneath most of the mathematical operations. The election constants can be configured, but there is a standard set which is recommended for most use cases. These can be found in `constants.py`.

**⚠️ Warning ⚠️**

There are some test constants used for testing code, but these are never to be used in any production system. The small and extra small constants are unlikely to be used except in rare cases for property testing due to collisions that will happen for smaller number sets.